<a href="https://colab.research.google.com/github/jimregan/tesseract-gle-uncial/blob/master/Update_gle_uncial_traineddata_for_Tesseract_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grab this for later

In [0]:
!wget https://github.com/jimregan/tesseract-gle-uncial/releases/download/v0.1beta2/gle_uncial.traineddata

Install dependencies

In [0]:
!apt-get install libicu-dev libpango1.0-dev libcairo2-dev libleptonica-dev


Clone, compile and set up Tesseract

In [8]:
!git clone https://github.com/tesseract-ocr/tesseract

Cloning into 'tesseract'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 34087 (delta 0), reused 3 (delta 0), pack-reused 34082
Receiving objects: 100% (34087/34087), 44.79 MiB | 9.99 MiB/s, done.
Resolving deltas: 100% (26460/26460), done.


In [0]:
import os
os.chdir('tesseract')

In [0]:
!sh autogen.sh

In [0]:
!./configure --disable-graphics


In [0]:
!make -j 8
!make install
!ldconfig
!make training
!make training-install

Grab some things to scrape the RIA corpus

In [17]:
import os
os.chdir('/content')
!git clone https://github.com/jimregan/tesseract-gle-uncial/

Cloning into 'tesseract-gle-uncial'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1402 (delta 5), reused 0 (delta 0), pack-reused 1373
Receiving objects: 100% (1402/1402), 200.19 MiB | 13.09 MiB/s, done.
Resolving deltas: 100% (634/634), done.
Checking out files: 100% (630/630), done.


In [0]:
!apt-get install lynx

Scrape the RIA corpus

In [0]:
! for i in A B C D E F G H I J K L M N O P Q R S T U V W X Y Z;do lynx -dump "http://corpas.ria.ie/index.php?fsg_function=1&fsg_page=$i" |grep http://corpas.ria.ie|awk '{print $NF}' >> list;done

In [0]:
!grep 'function=3' list |sort|uniq|grep corpas.ria|sed -e 's/function=3/function=5/' > input

In [0]:
!wget -x -c -i input

In [0]:
!mkdir text
!for i in corpas.ria.ie/*;do id=$(echo $i|awk -F'=' '{print $NF}');cat $i | perl /content/tesseract-gle-uncial/scripts/extract-ria.pl > text/$id.txt;done

Get the raw corpus in a single text file

In [0]:
!cat text/*.txt|grep -v '^$' > ria-raw.txt


Compress the raw text; this can be downloaded through the file browser on the left, so the scraping steps can be skipped in future

In [0]:
!gzip ria-raw.txt

...and can be re-added using the upload feature in the file browser

In [0]:
!gzip -d ria-raw.txt.gz

This next part is so I can update the langdata files

In [33]:
import os
os.chdir('/content')
!git clone https://github.com/tesseract-ocr/langdata

Cloning into 'langdata'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2164 (delta 9), reused 12 (delta 2), pack-reused 2129
Receiving objects: 100% (2164/2164), 400.19 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (366/366), done.
Checking out files: 100% (1022/1022), done.


In [38]:
!wget https://raw.githubusercontent.com/tesseract-ocr/tesseract/01cf7402dfb3e511a93a04376370ca927aaa0b48/contrib/genlangdata.pl

--2020-04-19 13:46:44--  https://raw.githubusercontent.com/tesseract-ocr/tesseract/01cf7402dfb3e511a93a04376370ca927aaa0b48/contrib/genlangdata.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6179 (6.0K) [text/plain]
Saving to: ‘genlangdata.pl’

genlangdata.pl      100%[===================>]   6.03K  --.-KB/s    in 0s      

2020-04-19 13:46:45 (45.1 MB/s) - ‘genlangdata.pl’ saved [6179/6179]



In [43]:
!wget https://raw.githubusercontent.com/jimregan/tesseract-gle-uncial/master/scripts/toponc.pl

--2020-04-19 14:02:07--  https://raw.githubusercontent.com/jimregan/tesseract-gle-uncial/master/scripts/toponc.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 563 [text/plain]
Saving to: ‘toponc.pl’

toponc.pl           100%[===================>]     563  --.-KB/s    in 0s      

2020-04-19 14:02:07 (34.2 MB/s) - ‘toponc.pl’ saved [563/563]



In [0]:
!cat ria-raw.txt | perl toponc.pl > ria-ponc.txt

In [0]:
!rm -rf genwlout

In [0]:
!perl genlangdata.pl -i ria-ponc.txt -d genwlout -p gle_uncial

In [0]:
os.chdir('genwlout')
!mv gle_uncial.word.punc.unsorted gle_uncial.punc.unsorted
!mv gle_uncial.word.numbers.unsorted gle_uncial.numbers.unsorted
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cat $i.unsorted /content/langdata/gle_uncial/$i | sort | uniq > $i;done

In [0]:
Grab the fonts

In [0]:
import os
os.chdir('/content')
!mkdir fonts
os.chdir('fonts')
!wget -i /content/tesseract-gle-uncial/fonts.txt

In [0]:
!for i in *.zip; do unzip $i;done